In [78]:
import sys
import pickle
import numpy
import pandas
import sklearn
import matplotlib 
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from numpy import mean

In [79]:
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from sklearn import cross_validation
from sklearn import datasets

In [80]:
'''Task 1: Select what features you'll use.
 features_list is a list of strings, each of which is a feature name.
 The first feature must be "poi".'''

'Task 1: Select what features you\'ll use.\n features_list is a list of strings, each of which is a feature name.\n The first feature must be "poi".'

In [53]:
features_list = ['poi','salary','to_messages','total_payments','exercised_stock_options','bonus',
'restricted_stock','shared_receipt_with_poi','restricted_stock_deferred','total_stock_value','expenses',
'loan_advances','from_messages','from_this_person_to_poi','poi','director_fees','deferred_income',
'from_poi_to_this_person']# You will need to use more features



In [77]:
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pandas.DataFrame.from_records(list(data_dict.values()))
persons = pandas.Series(list(data_dict.keys()))
persons

0                METTS MARK
1             BAXTER JOHN C
2            ELLIOTT STEVEN
3          CORDES WILLIAM R
4            HANNON KEVIN P
5       MORDAUNT KRISTINA M
6          MEYER ROCKFORD G
7           MCMAHON JEFFREY
8          HORTON STANLEY C
9           PIPER GREGORY F
10          HUMPHREY GENE E
11           UMANOFF ADAM S
12        BLACHMAN JEREMY M
13             SUNDE MARTIN
14             GIBBS DANA R
15          LOWRY CHARLES P
16           COLWELL WESLEY
17            MULLER MARK S
18       JACKSON CHARLENE R
19       WESTFAHL RICHARD K
20         WALTERS GARETH W
21        WALLS JR ROBERT H
22           KITCHEN LOUISE
23              CHAN RONNIE
24            BELFER ROBERT
25       SHANKMAN JEFFREY A
26            WODRASKA JOHN
27     BERGSIEKER RICHARD P
28          URQUHART JOHN A
29          BIBI PHILIPPE A
               ...         
116       REYNOLDS LAWRENCE
117     DIMICHELE RICHARD G
118        BHATNAGAR SANJAY
119        CARTER REBECCA C
120       BUCHANAN H

In [55]:
df.replace(to_replace='NaN', value=numpy.nan, inplace=True)
print df.head()


       bonus  deferral_payments  deferred_income  director_fees  \
0   600000.0                NaN              NaN            NaN   
1  1200000.0          1295738.0       -1386055.0            NaN   
2   350000.0                NaN        -400729.0            NaN   
3        NaN                NaN              NaN            NaN   
4  1500000.0                NaN       -3117011.0            NaN   

              email_address  exercised_stock_options  expenses  from_messages  \
0      mark.metts@enron.com                      NaN   94299.0           29.0   
1                       NaN                6680544.0   11200.0            NaN   
2  steven.elliott@enron.com                4890344.0   78552.0            NaN   
3     bill.cordes@enron.com                 651850.0       NaN           12.0   
4    kevin.hannon@enron.com                5538001.0   34039.0           32.0   

   from_poi_to_this_person  from_this_person_to_poi        ...          \
0                     38.0          

In [56]:
print df.isnull().sum()


bonus                         64
deferral_payments            107
deferred_income               97
director_fees                129
email_address                 35
exercised_stock_options       44
expenses                      51
from_messages                 60
from_poi_to_this_person       60
from_this_person_to_poi       60
loan_advances                142
long_term_incentive           80
other                         53
poi                            0
restricted_stock              36
restricted_stock_deferred    128
salary                        51
shared_receipt_with_poi       60
to_messages                   60
total_payments                21
total_stock_value             20
dtype: int64


In [57]:
print df.shape

(146, 21)


In [58]:
### Task 2: Remove outliers
# Removing column from database if counter > 72

for column, series in df.iteritems():
    if series.isnull().sum() > 72:
        df.drop(column, axis=1, inplace=True)
# Drop email address column
if 'email_address' in list(df.columns.values):
    df.drop('email_address', axis=1, inplace=True)
df.head()

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,poi,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,600000.0,NaN,94299.0,29.0,38.0,1.0,1740.0,False,585062.0,365788.0,702.0,807.0,1061827.0,585062.0
1,1200000.0,6680544.0,11200.0,NaN,NaN,NaN,2660303.0,False,3942714.0,267102.0,NaN,NaN,5634343.0,10623258.0
2,350000.0,4890344.0,78552.0,NaN,NaN,NaN,12961.0,False,1788391.0,170941.0,NaN,NaN,211725.0,6678735.0
3,NaN,651850.0,NaN,12.0,10.0,0.0,NaN,False,386335.0,NaN,58.0,764.0,NaN,1038185.0
4,1500000.0,5538001.0,34039.0,32.0,32.0,21.0,11350.0,True,853064.0,243293.0,1035.0,1045.0,288682.0,6391065.0


In [59]:
### Task 3: Create new feature(s)
poi_ratio = (df['from_poi_to_this_person'] + df['from_this_person_to_poi']) / (df['from_messages'] + df['to_messages'])
fraction_to_poi = (df['from_this_person_to_poi']) / (df['from_messages'])
fraction_from_poi = (df['from_poi_to_this_person']) / (df['to_messages'])
scale = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 100), copy=True)

df['poi_ratio'] = pandas.Series(poi_ratio) * 100
df['fraction_to_poi'] = pandas.Series(fraction_to_poi) * 100
df['fraction_from_poi'] = pandas.Series(fraction_from_poi) * 100

df.replace(to_replace='NaN', value=numpy.nan, inplace=True)
df= df.replace(to_replace=numpy.nan, value=0)
df = df.fillna(0).copy(deep=True)
df.columns = list(df.columns.values)
salary_scaled = scale.fit_transform(df['salary'])
print salary_scaled       


[  1.36977555e+00   1.00022360e+00   6.40127075e-01   0.00000000e+00
   9.11065435e-01   1.00018990e+00   0.00000000e+00   1.38722597e+00
   0.00000000e+00   7.38051640e-01   4.89525461e-01   1.08068651e+00
   9.30736476e-01   9.64214320e-01   0.00000000e+00   0.00000000e+00
   1.08051051e+00   9.42375082e-01   1.08057042e+00   2.38703765e-01
   0.00000000e+00   1.33720768e+00   1.01647571e+00   0.00000000e+00
   0.00000000e+00   1.13880839e+00   0.00000000e+00   7.03716254e-01
   0.00000000e+00   7.99966927e-01   9.33189271e-01   0.00000000e+00
   8.66267287e-01   0.00000000e+00   6.82457449e-01   0.00000000e+00
   7.93087866e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   8.39960592e-01   1.02510355e+00   1.27054033e+00
   8.11040079e-01   7.88264660e-01   0.00000000e+00   0.00000000e+00
   1.02186062e+00   1.78623393e-03   0.00000000e+00   1.00761569e+00
   1.60566328e+00   7.93297571e-01   0.00000000e+00   7.71866508e-01
   6.52503392e-01   1.91117295e+00

/home/aakarsh/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/aakarsh/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [60]:
print df.isnull().sum()
   

bonus                      0
exercised_stock_options    0
expenses                   0
from_messages              0
from_poi_to_this_person    0
from_this_person_to_poi    0
other                      0
poi                        0
restricted_stock           0
salary                     0
shared_receipt_with_poi    0
to_messages                0
total_payments             0
total_stock_value          0
poi_ratio                  0
fraction_to_poi            0
fraction_from_poi          0
dtype: int64


In [61]:
df.head()

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,poi,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,poi_ratio,fraction_to_poi,fraction_from_poi
0,600000.0,0.0,94299.0,29.0,38.0,1.0,1740.0,False,585062.0,365788.0,702.0,807.0,1061827.0,585062.0,4.665072,3.448276,4.708798
1,1200000.0,6680544.0,11200.0,0.0,0.0,0.0,2660303.0,False,3942714.0,267102.0,0.0,0.0,5634343.0,10623258.0,0.000000,0.000000,0.000000
2,350000.0,4890344.0,78552.0,0.0,0.0,0.0,12961.0,False,1788391.0,170941.0,0.0,0.0,211725.0,6678735.0,0.000000,0.000000,0.000000
3,0.0,651850.0,0.0,12.0,10.0,0.0,0.0,False,386335.0,0.0,58.0,764.0,0.0,1038185.0,1.288660,0.000000,1.308901
4,1500000.0,5538001.0,34039.0,32.0,32.0,21.0,11350.0,True,853064.0,243293.0,1035.0,1045.0,288682.0,6391065.0,4.921077,65.625000,3.062201


In [62]:
### Store to my_dataset for easy export below.
my_dataset = data_dict
print df.to_dict(orient="index")[0]
print df.keys()[0]
labels = df['poi'].copy(deep=True).astype(int).as_matrix()
features = (df.drop('poi', axis=1)).fillna(0).copy(deep=True).as_matrix()

{'salary': 365788.0, 'to_messages': 807.0, 'total_payments': 1061827.0, 'fraction_from_poi': 4.708798017348203, 'bonus': 600000.0, 'poi_ratio': 4.6650717703349285, 'total_stock_value': 585062.0, 'expenses': 94299.0, 'fraction_to_poi': 3.4482758620689653, 'exercised_stock_options': 0.0, 'from_messages': 29.0, 'other': 1740.0, 'from_this_person_to_poi': 1.0, 'poi': False, 'shared_receipt_with_poi': 702.0, 'restricted_stock': 585062.0, 'from_poi_to_this_person': 38.0}
bonus


In [81]:
def evaluate_clf(clf, features, labels, num_iters=1000, test_size=0.3):
    print clf
    accuracy = []
    precision = []
    recall = []
    first = True
    for trial in range(num_iters):
        features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, 
	test_size=0.3, random_state=42)  ### Extract features and labels from dataset for local testing
        clf.fit(features_train, labels_train)
        pred = clf.predict(features_test)
        accuracy.append(accuracy_score(labels_test, pred))
        precision.append(precision_score(labels_test, pred))
        recall.append(recall_score(labels_test, pred))
        if trial % 10 == 0:
            if first:
                sys.stdout.write('\nProcessing')
            sys.stdout.write('.')
            sys.stdout.flush()
            first = False

    print "done.\n"
    print "precision: {}".format(mean(precision))
    print "recall:    {}".format(mean(recall))
    print "accuracy:    {}".format(mean(accuracy))
    return mean(accuracy), mean(precision), mean(recall)

In [82]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

from sklearn.cluster import KMeans
k_clf = Pipeline(steps=[
        ('scaler', StandardScaler()),
('classifier',  KMeans(n_clusters=2, tol=0.001))])

In [83]:
evaluate_clf(k_clf, features, labels)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=2, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.001,
    verbose=0))])

Processing....................................................................................................done.

precision: 0.00193181818182
recall:    0.017
accuracy:    0.873227272727


(0.87322727272727296, 0.0019318181818181817, 0.017000000000000001)

In [66]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = Pipeline(steps=[
        ('scaler', StandardScaler()),
('classifier',  RandomForestClassifier(max_depth = 5,max_features = 'sqrt',n_estimators = 10, random_state = 42))])


In [67]:
evaluate_clf(rf_clf, features, labels)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

Processing....................................................................................................done.

precision: 1.0
recall:    0.2
accuracy:    0.909090909091


(0.90909090909090873, 1.0, 0.20000000000000004)

In [68]:
from sklearn.ensemble import AdaBoostClassifier
ab_clf = Pipeline(steps=[
        ('scaler', StandardScaler()),
('classifier',  AdaBoostClassifier(n_estimators=100))])

In [69]:
evaluate_clf(ab_clf, features, labels)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None))])

Processing....................................................................................................done.

precision: 0.375
recall:    0.6
accuracy:    0.840909090909


(0.84090909090909127, 0.375, 0.59999999999999976)

In [70]:
from sklearn.linear_model import LogisticRegression
l_clf = Pipeline(steps=[
        ('scaler', StandardScaler()),
('classifier', LogisticRegression(tol = 0.001, C = 10**-8, penalty = 'l2', random_state = 42))])

In [71]:
evaluate_clf(l_clf, features, labels)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', LogisticRegression(C=1e-08, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False))])

Processing....................................................................................................done.

precision: 0.25
recall:    0.6
accuracy:    0.75


(0.75, 0.25, 0.59999999999999976)

In [74]:
#selecting adaboost as the final algorithm due to most accurate and relevant results.
clf = ab_clf
my_features_list=list(df.columns.values)

In [75]:
pickle.dump(clf, open("../data/my_classifier.pkl", "w"))
pickle.dump(my_dataset, open("../data/my_dataset.pkl", "w"))
pickle.dump(my_features_list, open("../data/my_feature_list.pkl", "w"))